In [1]:
# activate line execution
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
from MulticoreTSNE import MulticoreTSNE as TSNE
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [4]:
import os
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf

initialize(version_base=None, config_path="configs")
cfg = compose(config_name="config.yaml")

hydra.initialize()

/mlodata1/hokarami/Machine-Learning-Collection/ML/Pytorch/GANs/4. WGAN-GP/test2/lib/python3.8/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)


In [5]:
cfg.cwgan

{'epochs': 150, 'cri_iter': 1, 'lr': 0.0003, 'bs': 128, 'img_size': '${data.img_size}', 'd_conditional': '${data.d_conditional}', 'cond': True, 'channels': 2, 'z_dim': 100, 'kernel_size': 4, 'feat_cri': 16, 'feat_gen': 16, 'lambda_gp': 10, 'lambda_corr': 0, 'lambda_l1': 0, 'wandb_project_name': 'CWGAN', 'device': '${device}'}

In [ ]:
from dotenv import load_dotenv
import wandb
import os

load_dotenv()
wandb.login(key=os.getenv("WANDB_KEY"))

api = wandb.Api()

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from tqdm import tqdm

# Load Data

In [ ]:
pd.options.mode.copy_on_write

In [ ]:
from data.data_utils import get_datasets

train_dataset, val_dataset = get_datasets(
        cfg.data, split=cfg.split, preprocess=True
    )

# Train/Load Model

In [ ]:
from Utils.models import TimEHR

model = TimEHR(cfg)

In [ ]:
# model.from_pretrained(path_cwgan='hokarami/CWGAN/uj5gf643',path_pix2pix='hokarami/PIXGAN/lorajx1i')
model.from_pretrained(path_cwgan='hokarami/CWGAN/234t1alr',path_pix2pix='hokarami/PIXGAN/9a0u31rl')


In [ ]:
model.train(train_dataset, val_dataset)

In [ ]:
model.cwgan.run_path
model.cwgan.epoch_no

model.pix2pix.run_path
model.pix2pix.epoch_no

model.progress

# Generation

In [ ]:
from Utils.utils import mat2df
# get synthetic data
fake_static, fake_data = model.generate(train_dataset, count=len(train_dataset), method='real')
df_ts_fake, df_static_fake = mat2df(fake_data,fake_static, train_dataset.dynamic_processor, train_dataset.static_processor)

# get train data
train_static, train_data = model._get_data(train_dataset)
df_ts_train, df_static_train = mat2df(train_data,train_static, train_dataset.dynamic_processor, train_dataset.static_processor)

# get test data
val_static, val_data = model._get_data(val_dataset)
df_ts_test, df_static_test = mat2df(val_data,val_static,val_dataset.dynamic_processor,val_dataset.static_processor)

In [ ]:
fake_data.shape

fake_data[0,1,:,:].sum()

In [ ]:
df_ts_fake

df_ts_train

# Evaluation

In [ ]:
from Utils.evaluation import evaluate

inputs = {

    # normalized data
    'fake_static': fake_static,
    'fake_data': fake_data,
    'train_static': train_static,
    'train_data': train_data,
    'test_static': val_static,
    'test_data': val_data,

    # dataframes from normalized data
    'df_ts_fake': df_ts_fake,
    'df_static_fake': df_static_fake,
    'df_ts_train': df_ts_train,
    'df_static_train': df_static_train,
    'df_ts_test': df_ts_test,
    'df_static_test': df_static_test,

    'state_vars': train_dataset.temporal_features,
}

In [ ]:
fake_data.shape, train_data.shape, val_data.shape

In [ ]:
wandb_task_name = f"NOTEBOOK-{cfg.data.name}-s{cfg.split}"

evaluate(inputs, wandb_task_name)

# You can now view the results in the wandb dashboard under USERNAME/TimEHR_Results
# The results are also available locally in Results/{wandb_task_name}/wandb/{run_name}/